## Clustering

### Import libraries 

In [1]:
# Import standard libraries
import os, datetime
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle as pl
import igraph
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import bokeh
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')
%matplotlib inline

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()
import seaborn as sb
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
# import cairo
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import zscore
from scipy.sparse.linalg import norm
import networkx as nx
from pyvis.network import Network

## Load environment variables from .env file and find project root
import sys
from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
# data_path = Path(PROJECT_ROOT, 'results', 'eyemap')
store_path = Path(PROJECT_ROOT, 'results','AOTU_Connectivity')
sys.path.append(str(PROJECT_ROOT.joinpath('src')))


from utils import olc_client
c = olc_client.connect(verbose=True)

from utils.celltype_conn_by_roi import CelltypeConnByRoi
from utils.celltype_conn_plotter import CelltypeConnPlotter
from utils.plotter import plot_cns, save_figure, get_skeletons, get_skeleton, get_meshes, get_mesh, show_figure
from utils.helper import slugify
from utils.neuron_bag import NeuronBag

# Import neuPrint specific libraries
from neuprint import Client, fetch_neurons, NeuronCriteria as NC, fetch_neurons, fetch_simple_connections, fetch_adjacencies, connection_table_to_matrix, merge_neuron_properties, NotNull, fetch_synapse_connections, fetch_neurons,fetch_primary_rois, fetch_all_rois, fetch_synapses, fetch_roi_hierarchy
from neuprint.utils import connection_table_to_matrix


Loading BokehJS ...

Connected to https://neuprint-cns.janelia.org[cns].
Client: neuprint-python v1.7.4
User: aishahamid201@gmail.com [readwrite]



In [2]:
# Other libraries
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform

# Fetch neuron types and connectivity

In [3]:
# Fetch all neurons and get their types
neurons_all_df,roi_all_df = fetch_neurons(NC(type=None)) # Gets all neuron metadata.
neurons_all_df

/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code/.pixi/envs/default/lib/python3.12/site-packages/neuprint/client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]
/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code/.pixi/envs/default/lib/python3.12/site-packages/neuprint/client.py:619: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


,bodyId,instance,type,pre,post,downstream,upstream,size,status,statusLabel,...,hemibrainType,matchingNotes,rootSide,superclass,receptorType,mancSerial,subclass,trumanHl,inputRois,outputRois
0,10001,DNp01(GF)_R,DNp01,1015,18582,4491,18582,1.069992e+11,Traced,Roughly traced,...,Giant Fiber,None,None,descending_neuron,None,NaN,lt,None,"[AMMC(R), CV, CV-unspecified, CentralBrain, Ce...","[AMMC(R), CV, CV-unspecified, CentralBrain, Ce..."
1,10002,OCG01d_L,OCG01d,1443,709,9165,709,8.601516e+09,Traced,Roughly traced,...,OCG01,None,None,visual_projection,None,NaN,None,None,"[CentralBrain, CentralBrain-unspecified, IPS(R...","[CentralBrain, CentralBrain-unspecified, IPS(R..."
2,10003,VCH_R,VCH,4425,27014,30020,27014,8.946596e+09,Traced,Prelim Roughly traced,...,VCH,None,None,visual_centrifugal,None,NaN,None,None,"[CRE(R), CentralBrain, CentralBrain-unspecifie...","[CentralBrain, CentralBrain-unspecified, IPS(L..."
3,10005,AOTU019_R,AOTU019,2837,31981,23423,31981,1.184775e+10,Traced,Roughly traced,...,AOTU019,None,None,cb_intrinsic,None,NaN,None,None,"[AOTU(R), CRE(R), CentralBrain, CentralBrain-u...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
4,10006,VS_L,VS,426,20321,2175,20321,2.051390e+10,Traced,Prelim Roughly traced,...,VS,None,None,visual_projection,None,NaN,None,None,"[CentralBrain, CentralBrain-unspecified, GNG, ...","[CentralBrain, CentralBrain-unspecified, GNG, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177478,1471062202,None,None,67,41,480,41,7.772304e+07,Anchor,Sensory Anchor,...,None,None,R,vnc_sensory,None,NaN,hair plate,None,"[LegNp(T2)(L), VNC, VNC-unspecified]","[LegNp(T2)(L), VNC]"
177479,1506165734,None,None,0,0,0,0,NaN,Orphan,Orphan hotknife,...,None,None,None,None,None,NaN,None,None,[],[]
177480,1529157851,None,None,0,0,0,0,1.923202e+07,Orphan,Orphan,...,None,None,None,None,None,NaN,None,None,[],[]
177481,1546043332,None,None,0,0,0,0,NaN,Anchor,Sensory Anchor,...,None,None,None,None,None,NaN,None,None,[],[]


In [4]:
all_celltypes = neurons_all_df['type'].unique()
all_celltypes


array(['DNp01', 'OCG01d', 'VCH', ..., 'IN13A073', 'IN19A135', 'IN01B088'],
      dtype=object)

In [5]:
# Fetch AOTU neurons and get their types
neurons_AOTU_df,roi_AOTU_df = fetch_neurons(NC(type='^AOTU.*'))
neurons_AOTU_df

,bodyId,instance,type,pre,post,downstream,upstream,size,status,statusLabel,...,hemibrainType,matchingNotes,rootSide,superclass,receptorType,mancSerial,subclass,trumanHl,inputRois,outputRois
0,10005,AOTU019_R,AOTU019,2837,31981,23423,31981,11847745920,Traced,Roughly traced,...,AOTU019,None,None,cb_intrinsic,None,None,None,None,"[AOTU(R), CRE(R), CentralBrain, CentralBrain-u...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
1,10031,AOTU041_R,AOTU041,2134,20586,13446,20586,5326291658,Traced,Roughly traced,...,AOTU041,None,None,cb_intrinsic,None,None,None,None,"[AOTU(L), AOTU(R), CRE(L), CRE(R), CentralBrai...","[AOTU(L), AOTU(R), CentralBrain, CentralBrain-..."
2,10070,AOTU019_L,AOTU019,2836,30714,23596,30714,11809707230,Traced,Roughly traced,...,AOTU019,None,None,cb_intrinsic,None,None,None,None,"[AOTU(L), CRE(L), CentralBrain, CentralBrain-u...","[AOTU(L), CRE(L), CentralBrain, CentralBrain-u..."
3,10148,AOTU041_L,AOTU041,2336,19960,14187,19960,5592462575,Traced,Roughly traced,...,AOTU041,None,None,cb_intrinsic,None,None,None,None,"[AOTU(L), AOTU(R), BU(L), CRE(L), CentralBrain...","[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr..."
4,10212,AOTU042_L,AOTU042,3579,10679,22690,10679,4312493196,Traced,Roughly traced,...,AOTU042,None,None,cb_intrinsic,None,None,None,None,"[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr...","[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,555803,AOTU024_R,AOTU024,814,2240,6989,2240,964400165,Traced,Roughly traced,...,AOTU024,None,None,cb_intrinsic,None,None,None,None,"[AOTU(R), ATL(L), ATL(R), CRE(R), CentralBrain...","[AOTU(R), ATL(L), ATL(R), CentralBrain, Centra..."
308,557095,AOTU008_L,AOTU008,392,2167,3198,2167,832073642,Traced,Roughly traced,...,AOTU008,ventrally projecting axonal arbors mcns,None,cb_intrinsic,None,None,None,None,"[AOTU(L), AOTU(R), CentralBrain, CentralBrain-...","[AOTU(L), CentralBrain, CentralBrain-unspecifi..."
309,565505,AOTU008_R,AOTU008,315,2727,2802,2727,502528277,Traced,Roughly traced,...,AOTU008,ventrally projecting axonal arbors mcns,None,cb_intrinsic,None,None,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
310,902249,AOTU038_R,AOTU038,111,459,704,459,278656951,Traced,Roughly traced,...,AOTU038,None,None,cb_intrinsic,None,None,None,None,"[AOTU(R), AVLP(R), CentralBrain, CentralBrain-...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."


In [6]:
AOTU_celltypes = neurons_AOTU_df['type'].unique()
AOTU_celltypes


array(['AOTU019', 'AOTU041', 'AOTU042', 'AOTU023', 'AOTU012', 'AOTU035',
       'AOTU005', 'AOTU100m', 'AOTU016_c', 'AOTU061', 'AOTU052',
       'AOTU103m', 'AOTU063_b', 'AOTU064', 'AOTU033', 'AOTU101m',
       'AOTU063_a', 'AOTU049', 'AOTU027', 'AOTU014', 'AOTU015', 'AOTU046',
       'AOTU050', 'AOTU024', 'AOTU059', 'AOTU009', 'AOTU008', 'AOTU045',
       'AOTU065', 'AOTU007_c', 'AOTU002_c', 'AOTU029', 'AOTU016_a',
       'AOTU002_a', 'AOTU026', 'AOTU020', 'AOTU017', 'AOTU028',
       'AOTU016_b', 'AOTU036', 'AOTU006', 'AOTU043', 'AOTU048', 'AOTU013',
       'AOTU051', 'AOTU034', 'AOTU011', 'AOTU002_b', 'AOTU022', 'AOTU062',
       'AOTU032', 'AOTU001', 'AOTU007_b', 'AOTU047', 'AOTU003',
       'AOTU007_a', 'AOTU054', 'AOTU053', 'AOTU030', 'AOTU056', 'AOTU055',
       'AOTU060', 'AOTU102m', 'AOTU038', 'AOTU058', 'AOTU037', 'AOTU018',
       'AOTU004', 'AOTU039', 'AOTU040', 'AOTU021', 'AOTU025'],
      dtype=object)

# Fetch synaptic connectivity

In [7]:
#  Define pre- and post-synaptic cell types
celltypes_pre = None
celltypes_post = None
# Fetch synaptic connectivity between these types
threshold = 3       # only connections with strength ≥ 3 are kept
neu_all_df, conn_all_df = fetch_adjacencies(celltypes_pre, celltypes_post, min_roi_weight=threshold) # a list of synaptic weights between pairs of neurons

  0%|          | 0/888 [00:00<?, ?it/s]

### Merge: combines metadata, bodyid, type, weights
#### simply constructing a weighted directed graph 𝐺=(𝑉,𝐸), where, V: neuron types, E: synapses with weights

In [ ]:
# Merge with neuron properties to get types on both sides, all neurons
conn_all_merge_df = merge_neuron_properties(neu_all_df, conn_all_df)


print(conn_all_merge_df.columns)

In [ ]:
print(conn_all_df.columns)

### Feature Vector Construction


In [ ]:
T = len(all_celltypes)
type_to_index = {ctype: idx for idx, ctype in enumerate(all_celltypes)}
type_to_index

#### feature vector of length 2T, First T values: incoming synaptic weights from all known types, Second T values: outgoing synaptic weights to all known types

In [ ]:
features = {}
for aotu in AOTU_celltypes:
    vec = np.zeros(2 * T)

    # Incoming connections to AOTU type
    incoming = conn_all_merge_df[conn_all_merge_df['type_post'] == aotu]
    for _, row in incoming.iterrows():
        pre = row['type_pre']
        if pre in type_to_index:
            vec[type_to_index[pre]] += row['weight']

    # Outgoing connections from AOTU type
    outgoing = conn_all_merge_df[conn_all_merge_df['type_pre'] == aotu]
    for _, row in outgoing.iterrows():
        post = row['type_post']
        if post in type_to_index:
            vec[T + type_to_index[post]] += row['weight']

    features[aotu] = vec

In [ ]:
incoming

In [ ]:
outgoing

#### Pairwise distance matrix

In [ ]:
# Step 1: Convert Feature Vectors to a Matrix

from scipy.spatial.distance import pdist
# Ensure AOTU cell types are in a consistent order
aotu_order = list(features.keys())

# Create feature matrix (rows: AOTU types, cols: 2T features)
feature_matrix = np.array([features[ctype] for ctype in aotu_order])

#### You compute Euclidean distance (L2 norm) between the connectivity fingerprints:

$$
d(i, j) = \| \vec{x}_i - \vec{x}_j \|_2 = \sqrt{ \sum_k (x_{ik} - x_{jk})^2 }
$$
This measures how different two AOTU types are based on their connectivity patterns.

In [ ]:
# Step 2: Compute Distances and Cluster

from scipy.cluster.hierarchy import linkage, fcluster

# Compute pairwise L2 (Euclidean) distances
distance_matrix = pdist(feature_matrix, metric='euclidean')


#### Hierarchial clustering 

In [ ]:
# Perform hierarchical clustering using average linkage
Z = linkage(distance_matrix, method='average')
# method='average' = UPGMA (Unweighted Pair Group Method with Arithmetic Mean)

# Optional: assign cluster labels by cutting dendrogram at height 0.3
clusters = fcluster(Z, t=4000, criterion='distance') # threshold = 4000
clusters

In [ ]:
# Step 3: Visualize with a Dendrogram
from scipy.cluster.hierarchy import dendrogram

plt.figure(figsize=(14, 6))
dendrogram(
    Z,
    labels=aotu_order,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=4000
)
plt.axhline(4000, color='red', linestyle='--')
plt.title("AOTU Cell Type Clustering by Connectivity")
plt.ylabel("L2 Distance")
plt.xlabel("AOTU Cell Types")
plt.tight_layout()
plt.show()

# Displays the hierarchy of neuron clusters. The y-axis = distance at which clusters merged.

In [ ]:
# Dynamically Inspect the Distance Scale 
print("Distance range:", distance_matrix.min(), "to", distance_matrix.max())
# Using this info to set threshold for distance for the previous step

print("Number of clusters:", len(np.unique(clusters)))

In [ ]:
# Step 4: Output Cluster Memberships

result_df = pd.DataFrame({
    'AOTU_cell_type': aotu_order,
    'cluster': clusters
})

# Print or save
print(result_df.sort_values('cluster'))
# result_df.to_csv("aotu_connectivity_clusters.csv", index=False)

## Visualize

In [ ]:
# we already have:
# - feature_matrix (shape: 72 x 2T)
# - clusters (length: 72)
# - aotu_order (list of AOTU types in feature_matrix order)

# Log-transform to improve contrast (avoid skew from large weights)
log_features = np.log1p(feature_matrix)  # log(1 + x)

# Wrap in a DataFrame for labeling
heatmap_df = pd.DataFrame(
    log_features,
    index=[f"{ctype} (C{cl})" for ctype, cl in zip(aotu_order, clusters)],
    columns=[f"in:{ctype}" for ctype in all_celltypes] +
            [f"out:{ctype}" for ctype in all_celltypes]
)


In [ ]:
plt.figure(figsize=(18, 12))
sb.heatmap(heatmap_df, cmap='viridis', xticklabels=False)
plt.title("AOTU Connectivity Heatmap (log-scaled synapse counts)")
plt.ylabel("AOTU Cell Types (clustered)")
plt.xlabel("Input / Output Cell Types")
plt.tight_layout()
plt.show()


## Cluster plot

MDS: Dimensionality reduction algorithm that tries to preserve pairwise distances.
Embeds high-dimensional points (here, 2T) into 2D for visualization.
> projecting neurons from a "connectivity space" to a "map space."

pdist() earlier gave a condensed distance vector (upper triangle of the distance matrix).
squareform() turns this into a full symmetric matrix:

$$
\text{distance\_sq}[i,j] = d(i,j)
$$

Shape:If you have 𝑁 AOTU types, distance_sq becomes an N × N matrix.

In [ ]:
from sklearn.manifold import MDS

# Convert distance vector to a squareform matrix
from scipy.spatial.distance import squareform
distance_sq = squareform(distance_matrix)


Apply Multidimensional Scaling (MDS)
Input: dissimilarity='precomputed' tells MDS to use the distance_sq matrix as-is.
Output: aotu_2d_coords: an N × 2 matrix where each row is the (x, y) coordinate of an AOTU type in 2D space.

In [ ]:
# Perform MDS to project into 2D
# Multidimensional scaling (MDS) projection 
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
aotu_2d_coords = mds.fit_transform(distance_sq)

In [ ]:
# Assign a color to each cluster
unique_clusters = np.unique(clusters)
palette = sb.color_palette("hsv", len(unique_clusters))
cluster_colors = [palette[np.where(unique_clusters == c)[0][0]] for c in clusters]

In [ ]:
plt.figure(figsize=(12, 8))
for i, (x, y) in enumerate(aotu_2d_coords):
    plt.scatter(x, y, color=cluster_colors[i], s=200, edgecolor='black')
    plt.text(x + 5, y, aotu_order[i], fontsize=12, ha='left', va='center')

plt.title("AOTU Cell Types Cluster Map (MDS from Euclidean Distance)")
plt.axis('on')
plt.tight_layout()
plt.show()

## Cluster plots

In [ ]:
from collections import defaultdict

# Group AOTUs by their cluster label
cluster_to_aotus = defaultdict(list)
for name, cl in zip(aotu_order, clusters):
    cluster_to_aotus[cl].append(name)

# Sort clusters (for consistent ordering)
sorted_cluster_labels = sorted(cluster_to_aotus.keys())

In [ ]:

# Flatten the ordered list of AOTU types by cluster groupings
ordered_aotus = []
aotu_cluster_labels = []
for cl in sorted_cluster_labels:
    aotus_in_cluster = cluster_to_aotus[cl]
    ordered_aotus.extend(aotus_in_cluster)
    aotu_cluster_labels.extend([cl] * len(aotus_in_cluster))

# Assign equally spaced angles
N = len(ordered_aotus)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False)

# Calculate (x, y) coordinates on the circle
radius = 10
x_coords = radius * np.cos(angles)
y_coords = radius * np.sin(angles)

In [ ]:
# Assign cluster colors
unique_clusters = sorted(np.unique(clusters))
palette = sb.color_palette("Paired", len(unique_clusters))
cluster_color_map = {cl: palette[i] for i, cl in enumerate(unique_clusters)}
node_colors = [cluster_color_map[cl] for cl in aotu_cluster_labels]

# Plot
plt.figure(figsize=(10, 10))
for i in range(N):
    plt.scatter(x_coords[i], y_coords[i], color=node_colors[i], s=200, edgecolor='black', zorder=3)
    plt.text(
        x_coords[i] * 1.15, y_coords[i] * 1.15, ordered_aotus[i],
        fontsize=10, ha='center', va='center', rotation=np.degrees(angles[i]),
        rotation_mode='anchor'
    )

# Optional: draw edges between adjacent same-cluster nodes
for i in range(N):
    if aotu_cluster_labels[i] == aotu_cluster_labels[(i + 1) % N]:
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[(i + 1) % N], y_coords[(i + 1) % N]
        plt.plot([x1, x2], [y1, y2], color='gray', alpha=1, zorder=1)

# Add cluster labels at mean angle for each cluster
for cl in sorted_cluster_labels:
    idxs = [i for i, c in enumerate(aotu_cluster_labels) if c == cl]
    mean_angle = np.mean([angles[i] for i in idxs])
    label_x = radius * 1.6 * np.cos(mean_angle)
    label_y = radius * 1.6 * np.sin(mean_angle)
    plt.text(label_x, label_y, f"Cluster {cl}", fontsize=12, weight='bold', ha='center', va='center', color=cluster_color_map[cl])

# Final formatting
plt.axis('off')       # Hides both axes and bounding box
plt.gca().set_aspect('equal', adjustable='datalim')  # Preserves aspect ratio without showing axes
plt.title("AOTU Cell Types Circular Cluster Plot", pad=60, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist
from collections import defaultdict

# --- Setup
aotu_order = list(features.keys())
feature_matrix = np.array([features[ctype] for ctype in aotu_order])
distance_matrix = pdist(feature_matrix, metric='euclidean')
Z = linkage(distance_matrix, method='average')
clusters = fcluster(Z, t=4000, criterion='distance')

# --- Prepare cluster mapping
cluster_to_aotus = defaultdict(list)
for name, cl in zip(aotu_order, clusters):
    cluster_to_aotus[cl].append(name)

sorted_cluster_labels = sorted(cluster_to_aotus.keys())
ordered_aotus = []
aotu_cluster_labels = []
for cl in sorted_cluster_labels:
    aotus = cluster_to_aotus[cl]
    ordered_aotus.extend(aotus)
    aotu_cluster_labels.extend([cl] * len(aotus))

# Circular layout
N = len(ordered_aotus)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False)
radius = 10
x_coords = radius * np.cos(angles)
y_coords = radius * np.sin(angles)
unique_clusters = sorted(np.unique(clusters))
palette = sb.color_palette("Paired", len(unique_clusters))
cluster_color_map = {cl: palette[i] for i, cl in enumerate(unique_clusters)}
node_colors = [cluster_color_map[cl] for cl in aotu_cluster_labels]

# --- Create subplot layout
fig, axes = plt.subplots(2, 1, figsize=(12, 14))

# --- Top panel: Standard dendrogram
dendrogram(
    Z,
    labels=aotu_order,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=4000,
    ax=axes[0]
)
axes[0].axhline(4000, color='red', linestyle='--')
axes[0].set_title("AOTU Cell Type Clustering by Connectivity")
axes[0].set_ylabel("L2 Distance")
axes[0].set_xlabel("AOTU Cell Types")

# --- Bottom panel: Circular cluster plot
axes[1].set_aspect('equal')
axes[1].axis('off')
for i in range(N):
    axes[1].scatter(x_coords[i], y_coords[i], color=node_colors[i], s=200, edgecolor='black', zorder=3)
    axes[1].text(
        x_coords[i] * 1.15, y_coords[i] * 1.15, ordered_aotus[i],
        fontsize=10, ha='center', va='center', rotation=np.degrees(angles[i]),
        rotation_mode='anchor'
    )

# Optional: draw edges
for i in range(N):
    if aotu_cluster_labels[i] == aotu_cluster_labels[(i + 1) % N]:
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[(i + 1) % N], y_coords[(i + 1) % N]
        axes[1].plot([x1, x2], [y1, y2], color='gray', alpha=1, zorder=1)

# Label clusters
for cl in sorted_cluster_labels:
    idxs = [i for i, c in enumerate(aotu_cluster_labels) if c == cl]
    mean_angle = np.mean([angles[i] for i in idxs])
    label_x = radius * 1.6 * np.cos(mean_angle)
    label_y = radius * 1.6 * np.sin(mean_angle)
    axes[1].text(label_x, label_y, f"Cluster {cl}", fontsize=12, weight='bold', ha='center', va='center', color=cluster_color_map[cl])

axes[1].set_title("AOTU Cell Types Circular Cluster Plot", pad=60, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
## Example for radialtree

import scipy.cluster.hierarchy as sch


import sys
sys.path.append('/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code/radialtree/radialtree')

import radialtree as rt

import numpy as np

# Create sample data and compute linkage
X = np.random.rand(5, 4)
Y = sch.linkage(X, method='single')
Z = sch.dendrogram(Y, no_plot=True)

# Plot it
rt.plot(Z)

